<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/cz_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install unidecode

In [15]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule
from unidecode import unidecode

def csv_files(path):
    """Função usada para ler os ficheiros que tem o simbolo do Euro"""
    import csv
    f = open(path, encoding='windows-1252', errors='ignore')
    data = []
    for row in csv.reader(f, delimiter=','):
        data.append(row)
    col = [*data[0]]
    data.pop(0)
    df = pd.DataFrame(data, columns=col)
    return df, col

def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index):
    """
    Params:\n
    path: parth of file in the computer.\n
    n_skiprows: Number of rows to delete in the original file,.\n
    columns_to_convert: Columns to convert the data general type. \n
    n_skipcolumn: Columns to skip in the original file. \n
    endrow = pass 0 to read everything, 1 to count entire
    columns_order: List of columns names in specific order to pass in the engine.\n
    """
    if date_format == '-':
        format = lambda x: datetime.strptime(x, "%d-%m-%Y")
    else:
        format = lambda x: datetime.strptime(x, "%d/%m/%Y")

    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows).fillna('')

    df = df.iloc[:,n_skip_columns:]

    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    """cont = 0
    for i in df.iloc[:,site_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[:cont, :]
    #df.columns = columns_order"""
    
    # convert intery columns to integer 
    #df[columns_integer_convert] = df[columns_integer_convert].fillna(0)
    #df[columns_integer_convert] = df[columns_integer_convert].astype('int64')

    return df

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def check_df(df, error_msg):
    if df == None or df.empty:
        return f'{error_msg}'
    else:
        return df

def count_duplicates(lista):
    count_dict = {}
    for entry in lista:
        if entry in count_dict.keys():
            count_dict[entry] += 1
        else:
            count_dict[entry] = 1
    
    duplicates = {}
    for k, v in count_dict.items():
        if v > 1:
            duplicates[k] = v
    return pd.DataFrame.from_dict(duplicates, orient='index', columns=['# of Duplicates'])

def defining_df(df, column_range, number_col):
    cont = 0
    for i in df.iloc[:,number_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[0:cont, :]
    return df

#old version
def check_columns(table, output_columns):
    """
    Check the total of number of missing columns and the missing columns in passed table.\n

    Params:\n
    table: contain the columns to be check.\n
    output_columns: columns structure at the final file. 

    Returns:\n
    Number of missing columns and a list that contains the name os missing columns.
    """
    total_received = len(table.columns)
    number_missing_columns = 0
    missing_columns = []
    #Counting of missing columns       
    #if country in contries:      
    for columns in output_columns:
        if columns.lower() not in [labels.lower() for labels in table]:
            number_missing_columns +=1
            missing_columns.append(columns)
    
    return total_received, number_missing_columns, missing_columns

def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

def replace_values(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
    #lista = []

    df.fillna('', inplace=True)
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

    return df

def date_parser(df, columns, t=1, format=1, type_dates='normal'):
    t_col = type_dates.lower()
    if format == 1:
        type_date = "%d/%m/%Y"
    else:
        type_date = "%d-%m-%Y"
    for column in columns:
        if t == 1:
            df[column] = pd.to_datetime(df[column], errors='coerce').fillna('')
            if t_col == 'mixed':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
        else:
            if t_col == 'mixed':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

# Refactorar esse codigo para receber todas as colunas num dic
# Sendo as keys=columns e values= picklist for each column
def check_date_columns(df, df_index,status_col,columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')

def check_amounts(df_check, df_index, status_col, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)
    
    filtered = df_check[[df_index, status_col]]

    for site in df[df_index]:
        for column in columns[1:]:
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        df_new = pd.merge(df_new, filtered, how='left', left_on='identification - site key', right_on=tw_index)
        df_new = df_new.set_index(tw_index)
        df_new = df_new[[status_col]+ df_new.columns[:-1].tolist()]
        #df_new = df_new[['identification - site key', status_col]]
        return df_new
    else: 
        print('\nNo one columns with incorrect Amount format!\n')
        
def check_picklist(df,df_index,df_status, df_cols, picklist_dict):

    df_picklist = df[df_cols]
    df_picklist['sites'] = df[df_index]
    df_picklist =  df_picklist.set_index('sites')
    
    #df_picklist = replace_values(df_picklist, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        columns = [i.lower() for i in picklist_dict.keys()]
        for column in set(columns): 
            value = str(df_picklist.loc[site,column])
            #print(value)
            if not value in set(picklist_dict[column]) or pd.isnull(value):
                #print(set(picklist_dict[column]))

                if not df_picklist.loc[site,df_index] in df_errors.index:
                    df_errors.loc[site,df_index] = df_picklist.loc[site,df_index]
                    
                    if pd.isnull(value) or pd.isna(value) or value == 'nan' or value == '':
                        df_errors.loc[site,column] = 'Blank Value'
                    else:
                        df_errors.loc[site,column] = f'Incorret picklist value: {value}'
                else:
                    
                    if pd.isnull(value) or pd.isna(value) or value == 'nan' or value == '':
                        df_errors.loc[site,column] = 'Blank Value'
                    else:
                        df_errors.loc[site,column] = f'Incorret picklist value: {value}'

    #df = df_errors.dropna()   
    df_errors = df_errors.dropna(how='all', axis=1).fillna('Ok!')
    if len(df_errors)>0:
        df = df[[df_index, df_status]]
        df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
        df_errors = df_errors.set_index(df_index)
        df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
        df_errors = df_errors.reset_index()
    else:
        print('\nNo one Picklist Error Founded!')
    return df_errors

def check_picklist_V1(df,df_index,df_status,picklist_dict):
    log = {}

    #picklist_dict={'Categorization by Transmission Sys':['Long-Term Mobile Sites','Macro','Non-Enterprise DAS','Repeater Sites','Public DAS Sites','Transmission']}
    for column in picklist_dict:
        df_aux=df.copy()
        #print(type(picklist_dict[column]))
        new=df_aux[column].isin(picklist_dict[column])
        #print(new)
        indexes=df.index[new == False].tolist()
        if column not in log:log[column]=[]
        log[column]=log[column]+indexes
    newDict = {}

    for key,value in log.items():
      for val in value:
          ID=df.iloc[val][df_index]
          if ID in newDict:
              v=f'Incorret picklist value: {df.iloc[val][key]}'
              if df.iloc[val][key]!=df.iloc[val][key] or pd.isnull(df.iloc[val][key]) or  pd.isna(df.iloc[val][key]) or df.iloc[val][key]=='' or df.iloc[val][key]=='nan':
                   v='Blank Value'
              newDict[ID][key]=v
          else:
              v=f'Incorret picklist value: {df.iloc[val][key]}'
              if df.iloc[val][key]!=df.iloc[val][key] or pd.isnull(df.iloc[val][key]) or  pd.isna(df.iloc[val][key]) or df.iloc[val][key]=='' or df.iloc[val][key]=='nan':
                   v='Blank Value'
              newDict[ID] = {'status': df.iloc[val][df_status],key:v}
    #print(newDict)
    logs = pd.DataFrame.from_dict(newDict, orient='index')
    logs.index.name='Site'
    logs = logs.dropna(how='all', axis=1).fillna('Ok!')
    return logs

def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [str(i) for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

    
    #list of new sites out of UIP sheet
    bts_out_uis = [str(i) for i in df_towerdb[df_towerdb[bts_col]=='Yes'][tw_index].sort_values() if i not in uip_list]
    bts_out_uis = pd.DataFrame(bts_out_uis, columns=['Bts_Sites_Out_UIS_File'])
    bts_out_uis = pd.merge(bts_out_uis, filtered, how='left', left_on=['Bts_Sites_Out_UIS_File'], right_on=tw_index)
    bts_out_uis = bts_out_uis[['Bts_Sites_Out_UIS_File', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]
    
    if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):
        return new_sites, bts_out_uis, df_site_bts[[tw_index,status_col, bts_col, bill_col]]
    else:
        print('\n No errors founded!\n')

def check_bts(df_tw, bts_tw_columns, tw_index, status_col, df_msa, bts_msa_column, msa_index):
    #Nested Function to make conditional validations
    def cond_bts_check(bts_msa, tw_bts_sites):
        bts_out_tw=[]
        if sorted(bts_msa) != sorted(tw_bts_sites):
            for i in tw_bts_sites:
                if i not in bts_msa:
                    bts_out_tw.append(i)

        return bts_out_tw

    bts_msa = msa[msa[bts_msa_column]=='Yes']
    bts_msa = [str(i) for i in bts_msa[msa_index]]

    tw_bts_sites = df_tw[df_tw[bts_tw_columns]=='Yes']
    tw_bts_sites = [str(i) for i in tw_bts_sites[tw_index]]

    #return of datas
    filtered = df_tw[[tw_index, status_col]]
    bts_out_tw = cond_bts_check(bts_msa, tw_bts_sites)
    df = pd.DataFrame(bts_out_tw, columns=['New Sites'])
    if not df.empty:
        df = pd.merge(df, filtered, how='left', left_on=['New Sites'], right_on=tw_index)
        df = df[['Bts_Sites_Out_UIS_File', status_col]]
        return df
    else: 
        print('\nNo errors founded!\n')

def check_wip(df_tw,tw_index, wip_tw, tw_bts, df_msa, msa_index, msa_wip):

    #Nested Function to make conditional validations
    def cond_wip_check(wip_msa, tw_wip_sites):
        count_wip = 0
        wip_out_tw=[]
        if sorted(wip_msa) != sorted(tw_wip_sites):
            for i in tw_wip_sites:
                if i not in wip_msa:
                    count_wip += 1
                    wip_out_tw.append(i)

        return wip_out_tw

    wip_msa = df_msa[df_msa[msa_wip]=='Yes']
    wip_msa = [str(i) for i in wip_msa[msa_index]]
    
    tw_wip_sites = df_tw[df_tw[wip_tw]=='Yes']
    tw_wip_sites = [str(i) for i in tw_wip_sites[tw_index]]
    
    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]
    
    wip_out_tw_list = cond_wip_check(wip_msa, tw_wip_sites)
    
    if not(len(wip_out_tw_list)==0 or tw_wip_site_bts_flagged.empty):
        return wip_out_tw_list, tw_wip_site_bts_flagged

    # Falta os outros países

def check_decommissioned(df,df_index,status_col, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    if not filtered.empty:
        return filtered[[df_index,status_col, decom_col, doer_col]]
    else:
        print('\nNo errors Founded!\n')
    
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    df_tw = df_tw[df_tw[status_col]==status][[tw_index, status_col, date_col]] 
    #print(df_tw)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[df_tw[date_col].astype('datetime64[ns]') < current_date]
            return filtered
        else:
            #bill_dates = pd.to_datetime(df_tw[date_col], errors==coerrce)
            filtered = df_tw[df_tw[date_col]=='']
            return filtered
    else:
        print('\nNo errors founded!\n')

def check_tw_doer_planned(df_tw, tw_index, doer_col,bill_col, status_col, dt_format):
    """Only GR until now"""
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date, format=dt_format)
    if not df_tw[doer_col].empty:
        df_tw[bill_col] = pd.to_datetime(df_tw[bill_col],errors='coerce', format=dt_format)
        filtered = df_tw[(df_tw[status_col]=='Planned')&(not df_tw[doer_col].astype('datetime64[ns]').empty)&\
                         (df_tw[bill_col].astype('datetime64[ns]') < current_date)]
        return filtered[[tw_index, status_col, bill_col, doer_col]]  
    else:
        print('\nNothing wrong in services sites!\n')
                                                              
def check_mom_bts(df_tw, tw_index,status_col, tw_col, df_msa, msa_index, msa_col):

    #c = country   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    if not filtered.empty:
        return filtered[[tw_index,status_col, tw_col]]    
    else:
        print('\nNo Errors Founded!\n')      

def check_lc_ta_dates(df,tw_index,status_col, start_date,end_date):
    filtered = df[df[start_date] > df[end_date]]
    if not filtered.empty:
        return filtered[[tw_index,status_col, start_date,end_date]]
    else:
        print('\nNo Errors Founded!\n')

def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites, country):
    t1 = ['pt', 'de', 'cz', 'ie', 'es', 'ro', 'hu']

    filtered = df_tw[[tw_index, status_tw_col]]
    if country.lower() in t1:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIS File!'])
        in_service_uip_sites = pd.merge(in_service_uip_sites, filtered, how='left', left_on='Site In service out of UIS File!',\
                                        right_on=tw_index)
        in_service_uip_sites = in_service_uip_sites[['Site In service out of UIS File!', status_tw_col]]
        
        #check for decomissioned site not in uip files
        if decom_col != "":
            tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
            decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
            decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                         right_on=tw_index)
            decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
        
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
            bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                         right_on=tw_index)
            bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
            critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                         right_on=tw_index)
            critical = critical[['Sites with critical level beyond 10% in out UIPS File', status_tw_col]]
            if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
                return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical
        else:
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
            bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                         right_on=tw_index)
            bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
            critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                         right_on=tw_index)
            critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
            if not (in_service_uip_sites.empty or bts_sites_out_uip.empty or critical.empty):
                return in_service_uip_sites, bts_sites_out_uip, critical
    else:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIS File!'])
        in_service_uip_sites = pd.merge(in_service_uip_sites, filtered, how='left', left_on='Site In service out of UIS File!',\
                                        right_on=tw_index)
        in_service_uip_sites = in_service_uip_sites[['Site In service out of UIS File!', status_tw_col]]
        
        #check for decomissioned site not in uip files
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index]]
        decomiss_sites_in_uip = []
        if set(tw_decomiss).intersection(uip_sites):
            decomiss_sites_in_uip = [i for i in tw_decomiss if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(decomiss_sites_in_uip, columns=['Decomissioned Site in UIS File'])
        decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                         right_on=tw_index)
        decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
        
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                         right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip = [i for i in df_uip['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Yes'][tw_index]
        critical = []
        if set(uip).intersection(bts_tw_critical):
            critical = [i for i in bts_tw_critical if i not in uip]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% out in UIS File'])
        critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                         right_on=tw_index)
        critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
        if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
            return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical

def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
        #lista = []

        df.fillna("", inplace=True)
        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df

    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    if not df_commercial_diffs.empty:
        return df_commercial_diffs
    else:
        print('\nNo Errors founded!')

def check_diffs_v2(path_current, path_last, cols_order, type_file='Excel', sheet='Commercial'):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)
    type_file = type_file.lower()
    #if type_file =='excel':
    _actual = pd.read_excel(path_current,sheet_name=sheet).fillna('')
    _before = pd.read_excel(path_last,sheet_name=sheet).fillna('')

    df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
    df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

    #df_final.style.apply(highlight_diff, axis=None)
    if not df_final.empty:
        return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)
    else:
        print('\nNo differences Founded!\n')

def general_log_erros(df_list, sheet_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save() 

def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save, type_file='mix',kind='tw', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, bill_cols):
        fit_cols = lower_str(list(df.columns))
        df.columns = fit_cols
        df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        #Ajustado só para CZ_Julho, tirei o encoding
        df_NEW = pd.read_csv(path_NEW).fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'mix':
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)
    else: 
        df_OLD = fit_df(path_OLD,bill_cols)
        df_NEW = fit_df(path_NEW,bill_cols)


    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]
                value_NEW = df_NEW.loc[row,col]
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
        status_1 = f'{status_col}_current'
        status_2 = f'{status_col}_before'
        new_copy = new_copy.set_index('sites')
        new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
        new_copy = new_copy.reset_index()


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} old_file vs new_file.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
     
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')



Comparing Files


In [ ]:
def find_diffs_between_files_1(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name,new_name, cols_conv, type_file='mix',status_col='', \
                             kind='tw', dates_cols=[], kind_col='',type_date="%d/%m/%Y", sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col,cols_conv, kind, kind_col):
        kind_col = kind_col.lower()
        kind = kind.lower()
        if kind == 'ta':
            df = df.dropna(subset=[index_col], axis=0)
            df['sites'] = df[index_col].astype(str) + df[kind_col] 
            #df.columns = lower_str(list(df.columns))
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            for col in cols_conv:
                df[col] = df[col].astype(str).apply(unidecode)
            df = df.dropna(subset=[index_col], axis=0)
            df = df.applymap(lambda x: x.encode('unicode_escape').decode('ISO-8859-1') if isinstance(x, str) else x)
            #df = df.drop_duplicates(subset=[index_col], keep='last')
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = str(df_OLD.loc[row,col])
                    value_NEW = str(df_NEW.loc[row,col])
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']]
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    def date_parser(df, columns, format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    cols_conv = lower_str(cols_conv)
    dates_cols = lower_str(dates_cols)
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        date_parser(df_OLD, dates_cols,format , type_date)
        df_OLD = fit_df(df_OLD,index_col,cols_conv,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        date_parser(df_NEW, dates_cols,format , type_date)
        df_NEW = fit_df(df_NEW,index_col,cols_conv,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD, encoding='ISO-8859-1').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        #df_OLD = df_OLD[lower_str(bill_cols)]
        df_OLD = fit_df(df_OLD, index_col, cols_conv,kind, kind_col)

        #cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW, encoding='ISO-8859-1').fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = fit_df(df_NEW, index_col,cols_conv, kind, kind_col)

    else:
        #cols_old = csv_header(path_OLD) header=0, names=cols_old,
        df_OLD = pd.read_csv(path_OLD,encoding='ISO-8859-1').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col,cols_conv, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        #print(df_NEW.columns)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        date_parser(df_NEW, dates_cols,format , type_date)
        df_NEW = fit_df(df_NEW, index_col,cols_conv, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

bill_col= ['Code (Duplicate)',\
            "Site Status",\
            "Categorization by Transmission Sys",\
            "Categorization by Site Type",\
            'Strategic Macro Sites',\
            'Critical Sites',\
            'Case A Core Site',\
            'Macro Site - Transmission Hub Site',\
            'Macro Site - Transmission Hub Site with/without Shelters',\
            'Transmission Sites',\
            'Room Configuration',\
            'Power Supply',\
            'Air Conditioning',\
            'Active Sharing Arrangements involving the Operator',\
            'VF-CZ Demerger phase',\
            "Billing Trigger date ",\
            'Strategic_Site_Bucket',\
            'Critical Site Bucket',\
            'Wip_Site',\
            'Bts_Site',\
            'Sites_As_Metered_Estimated',\
            'Subsequent_Sharing_Arrangement',\
            "First_Active_Sharing_Deployment_Type",\
            "First_Active_Sharing_Start_Date",\
            "First_Active_Sharing_End_Date",\
            "Decommissioned_Site",\
            "Transfer_Date_Of_Consent_Required_Sites",\
            "Date_Of_Equipment_Removal"]
pathtw = '/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx'
pathold = '/content/TowerDB_CzechRepublic_20210731.csv'
tw_save = '/content/CZ_TW'
tab = 'Tower DB'
row = 3
old_n = 'twdb_last.csv'
new_n = 'twdb_current.xlsx'
cols_con = ['Province','Municipality','Address']
dates = ['Billing Trigger date ','infrastructure ready (existing)/ to be ready (new)',\
         'First_Active_Sharing_Start_Date',\
         'First_Active_Sharing_End_Date', 'Transfer_Date_Of_Consent_Required_Sites',\
         'Date_Of_Equipment_Removal']  

find_diffs_between_files_1(pathold, pathtw, 'Code (Duplicate)', bill_col,tw_save, old_n,new_n,cols_con,'mix',\
                           status_col="Site Status" , kind='tw', dates_cols=dates , kind_col='', sheetname=tab, skipr = row)

In [17]:
# Read CZ towerdb files
def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index, cols_number, col_dates, format='normal', type_date="%d/%m/%Y"):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ']
        #lista = []

        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(0)
                else:
                    lista.append(index)
            df[column] = lista

        return df

    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    df = pd.read_excel(path, sheet_name = sheetname, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    date_parser(df, col_dates, format, type_date)
    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_number)

    for col in cols_number:
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')
    
    df = df.fillna('')

    return df

path_towerdb = '/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx'
towerdb_col = ["Code (Duplicate)","Site Status","VF - In scope / out of scope (Generalised scoping)","Site in Skylon scope Actual (From Site List Sheet )",\
               "Legacy Site Code(Duplicate)","TIMS Site Code","Legacy Site Code","Site Name","Macro Region","Province","Municipality","Inhabitants","Address",\
               "Ground Register","Altitude","Latitude","Longitude","Categorization by Inhabitants","Categorization by Transmission Sys",\
               "Categorization by Site Type","Categorization by Transmission Sys (subcluster)","Other internal Categorization 1 (Identify ACQ Sites)",\
               "Other internal Categorization 2 Energy provider (Eon/ LL)","DAS+Macro","DAS (Yes/ No)","DAS Ownership (Complete/ Partial/ 3rd Party)",\
               "Active/ Passive DAS","# of remote units/ radiating points","Type of Structure","Distance highest antenna to ground level","GBT Tower height",\
               "POD ID","Energy Consumption LTM (kwh)","Annual Energy cost LTM (Euros)","Infrastructure ready (existing)/ to be ready (new)",\
               "Infrastructure to be dismantled by","Radio equipments to be deactivated by","Infrastructure to be shared by","Technology VOD","Fibre / Microwave",\
               "Vertical passive structure owner","Room configuration (detailed)","Shelter passive structure ownership","Type of Air Conditioning",\
               "Number of cabinets (Full Capacity)","Number of Antenna (Full Capacity)","Number of MW (Full Capacity)","Counterpart","# of Lease Contracts",\
               "Current annual lease fees ","Current other fees (Maintenance)","Current other fees","(Average) residual duration - Lease contract",\
               "(Average) residual duration - Maintenance contract","(Average) residual duration - Lease & Maintenance both","# of Tenants Agreements",\
               "Current Total Annual Hosting Fees","Tenant (name/ID) MNO1 (Česká telekomunikační infrastruktura a.s.)","Annual Fee per Tenant MNO1",\
               "Annual Energy Fee MNO1","Annual Maintenance Fee MNO1","Other Services Fee MNO1","Residual duration MNO1 (Years)",\
               "Tenant (name/ID) MNO2 (T-Mobile Czech Republic a.s.)","Annual Fee per Tenant MNO2","Annual Energy Fee MNO2","Annual Maintenance Fee MNO2",\
               "Other Services Fee MNO2","Residual duration MNO2 (days)","Tenant (name/ID) MNO3","Annual Fee per Tenant MNO3","Annual Energy Fee MNO3",\
               "Annual Maintenance Fee MNO3","Other Services Fee MNO3","Residual duration MNO3","# of OTMOs","Annual Fee from OTMOs","Annual Energy Fee from OTMOs",\
               "Annual Maintenance Fee OTMOs","Other Services Fee OTMOs","Average residual duration (days)","Check","Strategic Macro Sites","Critical Sites",\
               "Case A Core Site","Macro Site - Transmission Hub Site","Macro Site - Transmission Hub Site with/without Shelters","Transmission Sites",\
               "Room Configuration","Power Supply","Air Conditioning","Active Sharing Arrangements involving the Operator","VF-CZ Demerger phase",\
               "EVO Location [FAR Site ID] ","Billing Trigger date ","Strategic_Site_Bucket","Critical Site Bucket","Subsequent_Sharing_Arrangement",\
               "First_Active_Sharing_Deployment_Type","First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date","Decommissioned_Site","Wip_Site",\
               "Bts_Site","Transfer_Date_Of_Consent_Required_Sites","Sites_As_Metered_Estimated","Date_Of_Equipment_Removal"]
towerdb_col = lower_str(towerdb_col)
dates = ['Billing Trigger date ', "Infrastructure ready (existing)/ to be ready (new)", "Infrastructure to be dismantled by",\
         'First_Active_Sharing_Start_Date','First_Active_Sharing_End_Date', 'Transfer_Date_Of_Consent_Required_Sites',\
         'Date_Of_Equipment_Removal']
num_parse = ["Inhabitants", "Current annual lease fees ","(Average) residual duration - Lease contract",\
             "(Average) residual duration - Maintenance contract","(Average) residual duration - Lease & Maintenance both",\
             "# of Tenants Agreements","Current Total Annual Hosting Fees", "Annual Fee per Tenant MNO1",\
             "Annual Energy Fee MNO1", "Annual Fee per Tenant MNO2","Annual Energy Fee MNO2",\
             "Residual duration MNO2 (days)", "Annual Fee from OTMOs","Annual Energy Fee from OTMOs",\
             "Other Services Fee OTMOs","Average residual duration (days)"]
tab = 'Tower DB'
row = 3
towerdb = read_files(path_towerdb,tab, 3, 0,"Code (Duplicate)",num_parse,dates)
towerdb.columns = lower_str(list(towerdb.columns))
towerdb = towerdb[towerdb_col]

path_msa = '/content/TowerDB_CzechRepublic_20210731 (3).csv'
msa = pd.read_csv(path_msa,encoding='ISO-8859-1').fillna('')
msa_cols = lower_str(list(msa.columns))
msa.columns = msa_cols

"""Check columns received"""
df_cols = check_columns_received(towerdb, msa_cols)
df_cols
#No colum


,Column(s) Missing
0,ï»¿code (duplicate)
1,tenant (name/id) mno1 (äeskã¡ telekomunikaän...


In [207]:
towerdb.to_excel('/content/TowerDB_CzechRepublic_20210831.xlsx', index=False)

In [218]:
towerdb.to_csv('/content/TowerDB_CzechRepublic_20210831.csv', index=False)

In [18]:
"""Defining variables which is gonna be reusable in checks"""
tw_index = "code (duplicate)"
tw_doer = "date_of_equipment_removal"
tw_status = "site status"
tw_bts = 'bts_site'
tw_bill = "billing trigger date "
tw_wip = 'wip_site'
tw_decom = 'decommissioned_site'
tw_critical = 'critical sites'

In [21]:
path_msa = '/content/TowerDB_CzechRepublic_20210731 (3).csv'
msa = pd.read_csv(path_msa,encoding='ISO-8859-1').fillna('')
cols_con = ['Province','Municipality','Address']
for col in cols_con:
   msa[col] = msa[col].astype(str).apply(unidecode)
msa = msa.applymap(lambda x: x.encode('unicode_escape').decode('ISO-8859-1') if isinstance(x, str) else x)
msa.columns = lower_str(towerdb_col)
msa.head(1)
#msa.columns = msa_cols


,code (duplicate),site status,vf - in scope / out of scope (generalised scoping),site in skylon scope actual (from site list sheet ),legacy site code(duplicate),tims site code,legacy site code,site name,macro region,province,municipality,inhabitants,address,ground register,altitude,latitude,longitude,categorization by inhabitants,categorization by transmission sys,categorization by site type,categorization by transmission sys (subcluster),other internal categorization 1 (identify acq sites),other internal categorization 2 energy provider (eon/ ll),das+macro,das (yes/ no),das ownership (complete/ partial/ 3rd party),active/ passive das,# of remote units/ radiating points,type of structure,distance highest antenna to ground level,gbt tower height,pod id,energy consumption ltm (kwh),annual energy cost ltm (euros),infrastructure ready (existing)/ to be ready (new),infrastructure to be dismantled by,radio equipments to be deactivated by,infrastructure to be shared by,technology vod,fibre / microwave,...,other services fee mno2,residual duration mno2 (days),tenant (name/id) mno3,annual fee per tenant mno3,annual energy fee mno3,annual maintenance fee mno3,other services fee mno3,residual duration mno3,# of otmos,annual fee from otmos,annual energy fee from otmos,annual maintenance fee otmos,other services fee otmos,average residual duration (days),check,strategic macro sites,critical sites,case a core site,macro site - transmission hub site,macro site - transmission hub site with/without shelters,transmission sites,room configuration,power supply,air conditioning,active sharing arrangements involving the operator,vf-cz demerger phase,evo location [far site id],billing trigger date,strategic_site_bucket,critical site bucket,subsequent_sharing_arrangement,first_active_sharing_deployment_type,first_active_sharing_start_date,first_active_sharing_end_date,decommissioned_site,wip_site,bts_site,transfer_date_of_consent_required_sites,sites_as_metered_estimated,date_of_equipment_removal
0,BADA1G,In Service,In-scope,Skylon,BADA1G,CZ-TIMS-41750,BADA1G,BKADA,East,JihomoravskA 1/2 kraj,Blansko,"4,590","Adamov, RonovskA! 4/443, 67904",600041,320,49.2917,16.6627,Suburban,Macro,RTT,Standard,No,Power Company-LV,No,No,0,0,,Rooftop structure,30.5,-,859182400200612435,11441,2073.185878,19/06/2000,,,,2G; 4G; 5G,Microwave,...,0.0,,,,,,,,0,0.0,0.0,,0.0,,,No,No,Non Core,No,Non Transmission Hub Site,No,Outdoor,DC,No,No Active Sharing,Phase 2,CZ62BADA1G,,Non Strategic,Non Critical,,,,,No,No,No,31/03/2023,Estimated Model,


In [ ]:
# Check Dates Format (dd-mm-YYYY)
"""Não deve haver NAN ou NAT na coluna, fazer fillna('') antes"""
def check_date_columns(df, df_index,status_col,bts_col, wip_col, columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col,bts_col, wip_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col,bts_col, wip_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')

def date_parser(df, columns, t=1, format=1, type_dates='normal'):
    t_col = type_dates.lower()
    if format == 1:
        type_date = "%d/%m/%Y"
    else:
        type_date = "%d-%m-%Y"
    for column in columns:
        if t == 1:
            df[column] = pd.to_datetime(df[column], errors='coerce').fillna('')
            if t_col == 'mixed':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
        else:
            if t_col == 'mixed':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

dates_cols = [tw_index,'Billing Trigger date ', 'First_Active_Sharing_Start_Date',\
              'First_Active_Sharing_End_Date', 'Transfer_Date_Of_Consent_Required_Sites',\
              'Date_Of_Equipment_Removal'] 
dates_cols = lower_str(dates_cols)

dates = ['Billing Trigger date ', 'First_Active_Sharing_Start_Date',\
              'First_Active_Sharing_End_Date', 'Transfer_Date_Of_Consent_Required_Sites',\
              'Date_Of_Equipment_Removal']           
dates = lower_str(dates)

#date_parser(towerdb, lower_str(dates), 2, 1, 'mixed')
#tw_in_service = tw_in_service[dates_cols]
tw_in_service = towerdb[towerdb['site status']=='In Service']
df_dates_errors = check_date_columns(tw_in_service,tw_index, tw_status, tw_bts, tw_wip, lower_str(dates_cols), 2)
df_dates_errors
#Errors em varias colunas

Validaton for general sites picklist

In [ ]:
dic_gen = [tw_index,'categorization by transmission sys']
pick_gen = {
    'categorization by transmission sys': ['Macro','Public DAS','Repeater','Transmission']
    }
df_pick_general_errors = check_picklist(towerdb, tw_index, tw_status, dic_gen, pick_gen)
df_pick_general_errors
#Pipeline sites without categorization

Validation for On air sites

In [ ]:
sites = list(towerdb[towerdb['air conditioning']=='Yes; Indoor Free air cooling / Free cooling units'][tw_index])


In [ ]:
def change_incorret(df, df_index, col_chg, incorrect_value, new_value):
    list_sites = list(df[df[col_chg]==incorrect_value][tw_index])
    df.loc[df[df_index].isin(list_sites), col_chg] = new_value
    print(df.loc[df[df_index].isin(list_sites), col_chg])

change_incorret(towerdb, tw_index, 'subsequent_sharing_arrangement', 'NO',\
                'No')

In [ ]:
picklist_cz = {
    'categorization by transmission sys': ['Macro','Public DAS','Repeater','Transmission'],
    'categorization by site type':['DAS passive','GBT','RTT'],
    'strategic macro sites':['Yes','No'],
    'critical sites':['Yes','No'],
    'case a core site':['Non Core', 'Case A','Case B'],
    'macro site - transmission hub site':['Yes','No'],
    'macro site - transmission hub site with/without shelters': ['With shelters','Without shelters','Non Transmission Hub Site'],
    'transmission sites': ['No','Yes'],
    'room configuration': ['Indoor','Outdoor',"Indoor & Outdoor"],
    'power supply':['AC','DC','AC & DC','No Power'],
    'air conditioning': ['No','Yes; Indoor Air Conditioning','Yes; Indoor Free Air cooling / Free cooling units'],
    'active sharing arrangements involving the operator':['MORAN (On VF equipment)','MORAN (On non-VF equipment)','MOCN with Spectrum Pooling','Partial Active-Passive','No Active Sharing'],
    'vf-cz demerger phase': ['Cancelled','Phase 1','Phase 2','Skylon BTS'],
    'strategic_site_bucket': ['Yes - 0-5%','Yes - 5-10%','Non Strategic'],
    'critical site bucket': ['Beyond 10%','Within 10%','Non Critical'],
    'subsequent_sharing_arrangement': ['Yes','No'],
    'decommissioned_site': ['Yes','No'],
    'wip_site': ['Yes','No'],
    'bts_site': ['Yes','No'],
    'sites_as_metered_estimated': ['Estimated Model','Metered Model']
}
dic_cols = [tw_index,'categorization by transmission sys',\
            'categorization by site type',\
            'strategic macro sites',\
            'critical sites',\
            'case a core site',\
            'macro site - transmission hub site',\
            'macro site - transmission hub site with/without shelters',\
            'transmission sites',\
            'room configuration',\
            'power supply',\
            'air conditioning',\
            'active sharing arrangements involving the operator',\
            'vf-cz demerger phase',\
            'strategic_site_bucket',\
            'critical site bucket',\
            'subsequent_sharing_arrangement',\
            'decommissioned_site',\
            'wip_site',\
            'bts_site',\
            'sites_as_metered_estimated']
df_pick_actives_errors = check_picklist(tw_in_service, tw_index, tw_status, dic_cols, picklist_cz)
df_pick_actives_errors
#Erros em varias colunas

Replace - Verificar real necessidade

In [ ]:
#check specific columns to change 'N/A', 0, and '-' to blank values 
def replace_values_v1(df, columns, value=""):
    invalid_values = ['N/A', 'n/a',"0", '-',0, '_', np.nan,'nan']
    #lista = []

    #df.fillna(0, inplace=True)
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        df[column] = df[column].fillna(0)
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

towerdb_bill_cols = ['Code (Duplicate)', 'Categorization by Transmission Sys', 'Categorization by Site Type',"Strategic Macro Sites",\
           "Critical Sites","Case A Core Site","Macro Site - Transmission Hub Site","Macro Site - Transmission Hub Site with/without Shelters",\
           "Transmission Sites","Room Configuration","Power Supply","Air Conditioning","Active Sharing Arrangements involving the Operator",\
           "VF-CZ Demerger phase","Billing Trigger date ","Strategic_Site_Bucket","Critical Site Bucket","Subsequent_Sharing_Arrangement",\
           "First_Active_Sharing_Deployment_Type","First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date","Decommissioned_Site",\
           "Wip_Site","Bts_Site","Transfer_Date_Of_Consent_Required_Sites","Sites_As_Metered_Estimated","Date_Of_Equipment_Removal"]
towerdb_bill_cols = lower_str(towerdb_bill_cols)
 
replace_values_v1(towerdb, towerdb_bill_cols, value="")

In [ ]:
df = towerdb

Month-on-Month BTS sites

In [9]:
def check_mom_bts(df_tw, tw_index,status_col, tw_col, df_msa, msa_index, msa_col):

    #c = country   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    if not filtered.empty:
        return filtered[[tw_index,status_col, tw_col]]    
    else:
        print('\nNo Errors Founded!\n') 
        
tw_filtered = towerdb[towerdb['site status']=='In Service']

df_mom_bts = check_mom_bts(tw_filtered, tw_index, tw_status, tw_bts, msa, tw_index, tw_bts)
df_mom_bts

df_mom_decom = check_mom_bts(tw_filtered, tw_index,tw_status, tw_decom, msa, tw_index, tw_decom)
df_mom_decom
# No errors in Both


No Errors Founded!


No Errors Founded!



In [29]:
#Read UIP File
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel('/content/UserInput_CzechRepublic_20210630 TrueUp.xlsx',sheet_name='SiteLevel',usecols=[0,2,3],skiprows=3)
uip.columns = uip_names

msa_sites = [i for i in msa[tw_index]]
tw_sites = [i for i in towerdb[tw_index]]
uip_sites = [i for i in uip['Site_ID']]



Verificar Errors de Sites Numericos com String

Verificar Errors de Sites Numericos com String

In [156]:
def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

    
    #list of new sites out of UIP sheet
    bts_out_uis = [i for i in df_towerdb[df_towerdb[bts_col]=='Yes'][tw_index] if i not in uip_list]
    bts_out_uis = pd.DataFrame(bts_out_uis, columns=['Bts_Sites_Out_UIS_File'])
    bts_out_uis = pd.merge(bts_out_uis, filtered, how='left', left_on=['Bts_Sites_Out_UIS_File'], right_on=tw_index)
    bts_out_uis = bts_out_uis[['Bts_Sites_Out_UIS_File', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    #print(current_date)
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]
    

    return new_sites, bts_out_uis, df_site_bts[[tw_index,status_col, bts_col, bill_col]]

new_sites, bts_sites_out_uis, df_bts_demerged = check_new_sites(towerdb, tw_index, tw_bts, tw_bill, tw_status, msa_sites, tw_sites, uip_sites)


In [ ]:
towerdb[towerdb[tw_index]==750210]

In [ ]:
new_sites

In [ ]:
# 13 sites
bts_sites_out_uis

In [73]:
df_wip_errors = check_wip(towerdb, tw_index, tw_wip, tw_bts, msa, tw_index, tw_wip)
df_wip_errors


No Errors Founded!



Validation DOER sites

In [114]:
#Validate Decomissioned Sites
def check_decommissioned(df,df_index,status_col, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    if not filtered.empty:
        return filtered[[df_index,status_col, decom_col, doer_col]]
    else:
        print('\nNo errors Founded!\n')

df_decom_errors = check_decommissioned(towerdb,tw_index, tw_status, tw_decom, tw_doer)
df_decom_errors


No errors Founded!



DOER Valitadions

In [115]:
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    df_tw = df_tw[df_tw[status_col]==status][[tw_index, status_col, date_col]] 
    #print(df_tw)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[df_tw[date_col].astype('datetime64[ns]') < current_date]
            return filtered
        else:
            #bill_dates = pd.to_datetime(df_tw[date_col], errors==coerrce)
            filtered = df_tw[df_tw[date_col]=='']
            return filtered
    else:
        print('\nNo errors founded!\n')
actives  = towerdb[towerdb[tw_status]=='In Service']
#Validate sites in service which has DOER less than current date

df_doer_errors = check_tw_bill_doer(actives, tw_index, tw_doer, tw_status, 'In Service', 'doer')
df_doer_errors

,code (duplicate),site status,date_of_equipment_removal
3900,40710X,In Service,30/08/2020


TowerDb vs UIS validations

UIS In Month

In [41]:
"""Check UIP In Month ites matches with Towerdb sites"""
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites, country):

    filtered = df_tw[[tw_index, status_tw_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    uis_sites_not_in_towerdb = []
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]
    if uis_sites_not_in_towerdb:
        uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
        uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                        right_on=tw_index)
        uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files

    if decom_col != "":
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
        decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                        right_on=tw_index)
        decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
    
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                        right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                    df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
        
        critical = []
        if len(bts_tw_critical) > 0:
            if set(uip_critical).intersection(bts_tw_critical):
                critical = [i for i in bts_tw_critical if i not in uip_critical]
                #print(critical)
        if critical:
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
            critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                        right_on=tw_index)
            critical = critical[['Sites with critical level beyond 10% in out UIPS File', status_tw_col]]
        #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
        return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical
    else:
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                        right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                    df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
        critical = []
        if len(uip_critical) > 0:
            critical = [i for i in bts_tw_critical if i not in uip_critical]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
        critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                        right_on=tw_index)
        critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
        #if not (in_service_uip_sites.empty or bts_sites_out_uip.empty or critical.empty):
        return in_service_uip_sites, bts_sites_out_uip, critical

uis_sites_not_in_towerdb,in_service_uis_sites, decomiss_sites_in_uis, bts_sites_out_uis, critical = check_uip_tw(towerdb,tw_index, tw_status, \
                                                                tw_decom, tw_bts,tw_critical, uip, uip_sites, 'cz')
#No errors

UIS True UP

In [34]:
"""Check UIP sites matches with Towerdb sites"""
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites, country):

    filtered = df_tw[[tw_index, status_tw_col]]
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    uis_sites_not_in_towerdb = []
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]
    if uis_sites_not_in_towerdb:
        uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS Sites True Up not active in TowerDB!'])
        uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS Sites True Up not active in TowerDB!',\
                                        right_on=tw_index)
        uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS Sites True Up not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS True Up Sites!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS True Up Sites!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS True Up Sites!', status_tw_col]]
    #check for decomissioned site not in uip files

    if decom_col != "":
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
        decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                        right_on=tw_index)
        decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
    
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                        right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                    df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
        
        critical = []
        if len(bts_tw_critical) > 0:
            if set(uip_critical).intersection(bts_tw_critical):
                critical = [i for i in bts_tw_critical if i not in uip_critical]
                #print(critical)
        if critical:
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
            critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                        right_on=tw_index)
            critical = critical[['Sites with critical level beyond 10% in out UIPS File', status_tw_col]]
        #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
        return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical
    else:
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                        right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                    df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
        critical = []
        if len(uip_critical) > 0:
            if set(uip_critical).intersection(bts_tw_critical):
                critical = [i for i in bts_tw_critical if i not in uip_critical]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
        critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                        right_on=tw_index)
        critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
        #if not (in_service_uip_sites.empty or bts_sites_out_uip.empty or critical.empty):
        return in_service_uip_sites, bts_sites_out_uip, critical


uis_sites_not_in_towerdb,in_service_uis_sites, decomiss_sites_in_uis, bts_sites_out_uis, critical = check_uip_tw(towerdb,tw_index, tw_status, \
                                                                tw_decom, tw_bts,tw_critical, uip, uip_sites, 'cz')
#No errors

Commercial Checks

In [ ]:
def check_diffs_v2(path_current, path_last, sheet='Commercial'):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)
    #type_file = type_file.lower()
    #if type_file =='excel':
    _actual = pd.read_excel(path_current,sheet_name=sheet).fillna('')
    _before = pd.read_excel(path_last,sheet_name=sheet).fillna('')

    df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
    df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

    #df_final.style.apply(highlight_diff, axis=None)
    if not df_final.empty:
        return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)
    else:
        print('\nNo differences Founded!\n')
path_current = '/content/UserInput_CzechRepublic_20210831.xlsx'
path_last = '/content/UserInput_CzechRepublic_20210731.xlsx'
df_comm_errors = check_diffs_v2(path_current, path_last,sheet='Commercial')
df_comm_errors
#No errors

In [136]:
def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
        #lista = []

        df.fillna("", inplace=True)
        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df

    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    #df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    if not df_commercial_diffs.empty:
        return df_commercial_diffs
    else:
        print('\nNo Errors founded!')

names = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', 'MSA Sites (Phase 1)\nInput_Value',\
        'PMA Sites (Phase 2)\nInput_Value','Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', \
              'Description/Instruction', 'Frequency of Update']
cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Data_Type','Input_Value_1_actual',\
                'Input_Value_1_before','Input_Value_2_actual','Input_Value_2_before''Equal Values','Description/Instruction', 'Frequency of Update']
# Check for commercial Values into current UIP File and compare with UIP File before

df_com = check_commercial(path_current, path_last,['MSA Sites (Phase 1)\nInput_Value','PMA Sites (Phase 2)\nInput_Value'], names, merge_cols, cols_ordered)
df_com
# No Errors


No Errors founded!


In [162]:
dfs = [df_dates_errors, df_pick_general_errors, df_pick_actives_errors, df_doer_errors]
names=['actives dates erros', 'Picklist General Errors', 'Actives Picklist Errors', 'In Service Site DOER(Past)']

general_log_erros(dfs, names, '/content/Towerdb_August_Errors.xlsx')

Create CSV File

Script to make TA Input file validation

In [ ]:
def find_diffs_between_files_1(path_OLD, path_NEW, index_col, bill_cols,path_save, old_name,new_name, cols_conv, type_file='mix',\
                               status_col='', kind='tw', dates_cols=[], kind_col='',type_date="%d/%m/%Y", sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col,cols_conv, kind, kind_col):
        kind_col = kind_col.lower()
        kind = kind.lower()
        if kind == 'ta':
            df = df.dropna(subset=[index_col], axis=0)
            df['sites'] = df[index_col].astype(str) + df[kind_col] 
            #df.columns = lower_str(list(df.columns))
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            if cols_conv:
                for col in cols_conv:
                    df[col] = df[col].astype(str).apply(unidecode)
            df = df.dropna(subset=[index_col], axis=0)
            df = df.applymap(lambda x: x.encode('unicode_escape').decode('ISO-8859-1') if isinstance(x, str) else x)
            #df = df.drop_duplicates(subset=[index_col], keep='last')
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = str(df_OLD.loc[row,col])
                    value_NEW = str(df_NEW.loc[row,col])
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']]
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    def date_parser(df, columns, format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    cols_conv = lower_str(cols_conv)
    dates_cols = lower_str(dates_cols)
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        date_parser(df_OLD, dates_cols,format , type_date)
        df_OLD = fit_df(df_OLD,index_col,cols_conv,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        date_parser(df_NEW, dates_cols,format , type_date)
        df_NEW = fit_df(df_NEW,index_col,cols_conv,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD, encoding='ISO-8859-1').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        #df_OLD = df_OLD[lower_str(bill_cols)]
        df_OLD = fit_df(df_OLD, index_col, cols_conv,kind, kind_col)

        #cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW, encoding='ISO-8859-1').fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = fit_df(df_NEW, index_col,cols_conv, kind, kind_col)

    else:
        #cols_old = csv_header(path_OLD) header=0, names=cols_old,
        ta_cols = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', \
           'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', \
           'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', \
           'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
           'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', \
           'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', \
           'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
           'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x','SiteType',\
           '1.028', 'Unique Tenant', 'Unique Tenant Count', 'unamed', 'unamed_2']
        df_OLD = pd.read_csv(path_OLD,header=0, names=ta_cols, encoding='ISO-8859-1').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        print(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col,cols_conv, kind, kind_col)
        
        col = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type',\
                'Contract start date', 'Contract end date', 'Status of contract',\
                'Renewal Option', 'Comments', 'Counterpart ID', 'Counterpart',\
                'Classification of Tenant', 'Annual amount - billing currency',\
                'Billing Currency', 'Annual Amount in CZK', 'Amount in EUR',\
                'Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
                'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key',\
                'Classification', 'Residual Period in Years', 'Remarks',\
                'Count of unique key', 'Count of contracts', 'Scoping classification',\
                'DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
                'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x',\
                'SiteType', '1.028', 'Unique Tenant', 'Unique Tenant Count','Quota of Unique Key','unamed']

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, header=0, names = col, skiprows = skipr)
        #print(df_NEW.columns)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        print(df_NEW.head(1))
        #df_NEW = df_NEW.reindex(columns=ta_cols)
        print(list(df_NEW.columns))
        df_NEW = df_NEW.reindex(columns=list(df_OLD.columns))
        df_NEW = df_NEW.fillna('')
        date_parser(df_NEW, dates_cols,format , type_date)
        df_NEW = fit_df(df_NEW, index_col,cols_conv, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    df_all.to_excel('/content/excel.xlsx', index=False)
    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

ta_cols = ['Tenant Agreement ID - NEW', 'Code','Contract start date', 'Contract end date','Counterpart',\
           'Classification of Tenant',]
path_ta_input = '/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx'
sheet = 'Tenant '
ta_old = '/content/TA_Input_CzechRepublic_20210731.csv'
ta_save = '/content/CZ_TA'
old_ta = 'TA_old.csv'
new_ta = 'TA_new.xlsx'
#find_diffs_between_files(ta_old, path_ta_input, 'Code', ta_cols, "", ta_save,'csv', 'ta')


cols_con_ta = ['Province','Municipality','Address']
dates = ['Contract start date', 'Contract end date', 'Update in month']

find_diffs_between_files_1(ta_old, path_ta_input, 'Code', ta_cols, ta_save, old_ta,new_ta, [],'mix',\
                           status_col="", kind='ta', dates_cols=dates , kind_col='Site Name', sheetname=sheet)


In [ ]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save, type_file='mix',kind='tw', kind_col='', k='', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, bill_cols):
        df = df.dropna(subset=[index_col], axis=0)
        #df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        #Ajustado só para CZ_Julho, tirei o encoding
        df_NEW = pd.read_csv(path_NEW).fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    else:
        ta_cols = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', \
           'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', \
           'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', \
           'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
           'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', \
           'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', \
           'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
           'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x','SiteType',\
           '1.028', 'Unique Tenant', 'Unique Tenant Count', 'unamed', 'unamed_2']
        ta_cols = lower_str(ta_cols)
        df_OLD = pd.read_csv(path_OLD, header=0, names=ta_cols, encoding='latin')
        #df_OLD = fit_df(df_OLD,bill_cols)
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = df_OLD.dropna(subset=[index_col], axis=0)
        #df_OLD['sites'] = [site if type(site) != str else len(df_OLD[index_col])+i for i, site in enumerate(df_OLD[index_col])]
        #df_OLD = df_OLD.sort_values(by='sites')
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        df_OLD['sites'] = df_OLD[index_col].astype(str)+df_OLD[kind_col].astype(str)+df_OLD['site name']+df_OLD['count of contracts'].astype(str)+df_OLD['service type'].astype(str)
        

        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])

        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_OLD = df_OLD.set_index('sites').fillna('')

        col = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type',\
                'Contract start date', 'Contract end date', 'Status of contract',\
                'Renewal Option', 'Comments', 'Counterpart ID', 'Counterpart',\
                'Classification of Tenant', 'Annual amount - billing currency',\
                'Billing Currency', 'Annual Amount in CZK', 'Amount in EUR',\
                'Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
                'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key',\
                'Classification', 'Residual Period in Years', 'Remarks',\
                'Count of unique key', 'Count of contracts', 'Scoping classification',\
                'DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
                'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x',\
                'SiteType', '1.028', 'Unique Tenant', 'Unique Tenant Count','Quota of Unique Key','unamed']
        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname,header=0, names=col, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW.reindex(columns=ta_cols)
        df_NEW = df_NEW.dropna(subset=[index_col], axis=0)
        df_NEW['sites'] = df_NEW[index_col].astype(str)+df_NEW[kind_col].astype(str)+df_NEW['site name']+df_NEW['count of contracts'].astype(str)+df_NEW['service type'].astype(str)
        #print([i for i in list(df_NEW['sites']) if list(df_NEW['sites']).count(i)>1])

     
        #print(df_NEW.info())
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos

        try:
            df_NEW = df_NEW.set_index('sites').fillna('')
        except:
            print('falha')
        #print(list(df_NEW.columns)==(df_OLD.columns))

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:

        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]
                value_NEW = df_NEW.loc[row,col]
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
        status_1 = f'{status_col}_current'
        status_2 = f'{status_col}_before'
        new_copy = new_copy.set_index('sites')
        new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
        new_copy = new_copy.reset_index()


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} old_file vs new_file.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                             end_color='00FF0000', fill_type='solid')
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)

    #print(len(newRows))
    for site in new_copy[index_col]:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

find_diffs_between_files(ta_old, path_ta_input, 'Code', ta_cols, \
                             status_col='', path_save=ta_save, type_file='mix',kind='ta',kind_col='tenant agreement id - new',k='service type', sheetname=sheet)

In [326]:
#ta.groupby(ta['Code'].apply(type) != str).apply(lambda i: i.sort_values('Code')).reset_index(drop = True)
sites = []
for i in ta['Code']:
    if type(i) == str:
        sites.append(i)
    else:
        sites.append(int(i))
ta['sites'] = sites
#ta.groupby(ta['sites'].apply(type) != str).apply(lambda i: i.sort_values('sites')).reset_index(drop = True)

In [211]:
#Read TA Input File
def read_files(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int, cols_amount, bill_cols, \
               format='mix', type_date="%d/%m/%Y"):
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df
    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    df = pd.read_excel(path, sheet_name = sheetname,header=0, names = col_name, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_amount, 0)
    for col in cols_amount:
        #df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')
    for col in cols_int:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int)
    df = df.fillna('')
    replace_values(df, bill_cols)
    date_parser(df, cols_date, format, type_date)
    # define the entiry columns which doesn't have NaN 
    return df

def replace_values_ta(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ']
    #lista = []
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        df[column] = df[column].fillna(0)
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

col = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type',\
       'Contract start date', 'Contract end date', 'Status of contract',\
       'Renewal Option', 'Comments', 'Counterpart ID', 'Counterpart',\
       'Classification of Tenant', 'Annual amount - billing currency',\
       'Billing Currency', 'Annual Amount in CZK', 'Amount in EUR',\
       'Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
       'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key',\
       'Classification', 'Residual Period in Years', 'Remarks',\
       'Count of unique key', 'Count of contracts', 'Scoping classification',\
       'DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
       'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x',\
       'SiteType', '1.028', 'Unique Tenant', 'Unique Tenant Count','Quota of Unique Key','unamed']

#ta = pd.read_csv(path_ta_input, header=0, names=col)
dates = ['Contract start date', 'Contract end date', 'Update in month']
interger = ['Residual Period in Years','Count of unique key', 'Count of contracts', 'Unique Tenant Count',\
            'Quota of Unique Key']
amount = ['Annual Amount in CZK', 'Amount in EUR']
ta_bill_cols = ['Tenant Agreement ID - NEW','Contract start date','Contract end date',\
                'Counterpart','Classification of Tenant']
ta = read_files('/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx', 'Tenant ',col, 0, 0, 'Code', dates, interger,amount, ta_bill_cols)

ta_cols = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', \
           'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', \
           'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', \
           'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
           'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', \
           'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', \
           'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
           'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x','SiteType',\
           '1.028', 'Unique Tenant', 'Unique Tenant Count', 'unamed', 'unamed_2']

ta = ta.reindex(columns=ta_cols)

ta.head(1)



,Tenant Agreement ID - NEW,Code,Site Name,Service Type,Contract start date,Contract end date,Status of contract,Renewal Option,Comments,Counterpart ID,Counterpart,Classification of Tenant,Annual amount - billing currency,Billing Currency,Annual Amount in CZK,Amount in EUR,Terms of Payment,Frequency,Indexed YES/NO,Index,Percentage,VAT Subject YES/NO,Percentage (VAT),Unique Key,Classification,Residual Period in Years,Remarks,Count of unique key,Count of contracts,Scoping classification,DAS sites,DAS ownership,31-Mar-21,Update in month,Updated Item,Comment,Counterpart ID_1,Counterpart_1,x,SiteType,1.028,Unique Tenant,Unique Tenant Count,unamed,unamed_2
0,2500000008,150910,LIUSI,Lease,10/09/2020,31/12/9999,effective contract,No,,400004777,a-net Liberec,OTMO,100.0,CZK,100,3,30 days deferred payment,yearly,NO,-,-,YES,0.21,150910|400004777,OTMO,8094,,1,1,In-Scope - Skylon BTS,No,,,01/04/2020,new contract,,,handover protocol signed 10.9.2020,,RTT,,150910|a-net Liberec,1,,NaN


In [ ]:
ta['Tenant Agreement ID - NEW'].value_counts()

In [212]:
ta.to_excel('/content/TA_Input_CzechRepublic_20210831.xlsx', index=False)

In [219]:
ta.to_csv('/content/TA_Input_CzechRepublic_20210831.csv', index=False)

In [ ]:
ta_msa = pd.read_csv('/content/TA_Input_CzechRepublic_20210731.csv', encoding='latin2')
msa_cols = list(ta_msa.columns)
ta_cols = list(ta.columns)
df_cols = check_columns_received(ta, msa_cols)
df_cols

In [175]:
def check_lc_ta_dates(df,tw_index,start_date,end_date):
    df[start_date] = pd.to_datetime(df[start_date],errors='coerce')
    df[end_date] = pd.to_datetime(df[end_date],errors='coerce')
    filtered = df[df[start_date] > df[end_date]]
    if not filtered.empty:
        return filtered[[tw_index, start_date,end_date]]
    else:
        print('\nNo Errors Founded!\n')

df_ta_dates = check_lc_ta_dates(ta,'Code', 'Contract start date', 'Contract end date')
df_ta_dates


No Errors Founded!



In [176]:
ta['Classification of Tenant'].value_counts()

MNO     323
OTMO    295
Name: Classification of Tenant, dtype: int64

Script to read LC Input file


In [5]:
def read_files(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int, cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y"):
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df
    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    df = pd.read_excel(path, sheet_name = sheetname,header=0, names = col_name, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_amount, 0)
    for col in cols_amount:
        #df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')

    df = df.fillna('')
    replace_values(df, bill_cols)
    date_parser(df, cols_date, format, type_date)
    # define the entiry columns which doesn't have NaN 
    return df

def replace_values_ta(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ']
    #lista = []
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        df[column] = df[column].fillna(0)
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

col_names = ['Contract ID - NEW', 'FIN ID', 'Contract Crncy', 'Contract Type', 'Freq.', 'Freq. Unit', 'Indexation',\
         'Index upon request', 'Counterpart ID', 'Counterpart', 'LC Amount CZK\nyearly', 'Amount in EUR yearly (Actual)', \
         'Payment terms Code', 'VAT Subject', '% of VAT', 'Contr. Start date', 'Contr. 1st End', 'Contr. Term End', \
         'Sublease Consession', 'Renewal Option', 'Unique Key', 'Count of Key', 'Count of ContrBct ', \
         'Remarks - Consistency check', 'Residual Period in years', 'Scoping classification', 'DAS sites', \
         'DAS ownership', '31_12_9999', 'Unique Key1', 'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021', \
         'Check LC amount', 'Updated Item\nAmount yearly', 'Contr. 1st End as of 31.05.2021', \
         'Contr. 1st End as of 30.06.2021', 'Check Contr. 1st End', 'Updated Item\n1st End date', 'Comment', \
         'Comment date', 'Date']

#ta = pd.read_csv(path_ta_input, header=0, names=col)
dates_lc = ['Contr. Start date', 'Contr. 1st End', 'Contr. 1st End as of 31.05.2021', 'Contr. 1st End as of 30.06.2021']
interger_lc = []
amount_lc = ['LC Amount CZK\nyearly']
lc_bill_cols = ['Contract ID - NEW','Counterpart']
lc = read_files('/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx', 'Final data_Lease',col_names, 1, 0, 'FIN ID', \
                dates_lc, interger_lc,amount_lc, lc_bill_cols)

col_order_lc = ["Contract ID - NEW","FIN ID","Contract Crncy","Contract Type","Freq.","Freq. Unit","Indexation",\
                "Index upon request","Counterpart ID","Counterpart","LC Amount CZK\nyearly","Amount in EUR yearly (Actual)",\
                "Payment terms Code","VAT Subject","% of VAT","Contr. Start date","Contr. 1st End","Contr. Term End",\
                "Sublease Consession","Renewal Option","Unique Key","Count of Key","Count of ContrBct ",\
                "Remarks - Consistency check","Residual Period in years","Scoping classification","DAS sites",\
                "DAS ownership","31_12_9999","Unique Key1",'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021',\
                "Check LC amount","Updated Item\nAmount yearly",'Contr. 1st End as of 31.05.2021','Contr. 1st End as of 30.06.2021',\
                "Check Contr. 1st End","Updated Item\n1st End date","Comment","Comment date","Date"]
lc = lc[col_order_lc]
lc.head(2)

,Contract ID - NEW,FIN ID,Contract Crncy,Contract Type,Freq.,Freq. Unit,Indexation,Index upon request,Counterpart ID,Counterpart,LC Amount CZK\nyearly,Amount in EUR yearly (Actual),Payment terms Code,VAT Subject,% of VAT,Contr. Start date,Contr. 1st End,Contr. Term End,Sublease Consession,Renewal Option,Unique Key,Count of Key,Count of ContrBct,Remarks - Consistency check,Residual Period in years,Scoping classification,DAS sites,DAS ownership,31_12_9999,Unique Key1,LC amount as of 31.05.2021,LC amount as of 30.06.2021,Check LC amount,Updated Item\nAmount yearly,Contr. 1st End as of 31.05.2021,Contr. 1st End as of 30.06.2021,Check Contr. 1st End,Updated Item\n1st End date,Comment,Comment date,Date
0,1100000000,111530,CZK,Lease,3,Months,Free - Modifiable,yes,800222850,Sabadinová Marie,"90,176",3281.549127,Due 14days from invoice receipt date,no,,28/06/2002,28/06/2022,,"Yes, but announcement needed",Yes,1100000000|800222850,1,1,Mapped,2,In-scope,No,,T00,1100000000|800222850|T00,90176.97,90176.97,0.0,"JULY - no change, OK",28/06/2022,28/06/2022,0,"JULY - no change, OK",,,
1,1100000001,111470,CZK,Lease,3,Months,Free - Modifiable,yes,800199147,"SPOLECENSTVI VLASTNIKU JEDNOTEK DOMU C.P. 589,...","88,819",3232.135371,Due 14days from invoice receipt date,yes,0.21,25/09/2002,31/12/2025,,"Yes, but announcement needed",Yes,1100000001|800199147,1,1,Mapped,5,In-scope,No,,T00,1100000001|800199147|T00,88819.08,88819.08,0.0,"JULY - no change, OK",31/12/2025,31/12/2025,0,"JULY - no change, OK",,,


In [9]:
lc['LC Amount CZK\nyearly'].value_counts()

50,000     115
30,000     106
60,000     106
40,000      94
80,000      92
          ... 
89,930       1
122,984      1
49,000       1
400,000      1
54,073       1
Name: LC Amount CZK\nyearly, Length: 2562, dtype: int64

In [214]:
lc.to_excel('/content/LC_Input_CzechRepublic_20210831.xlsx', index=False)

In [217]:
lc.to_csv('/content/LC_Input_CzechRepublic_20210831.csv', index=False)

In [ ]:
def check_amounts_lc(df_check, df_index, columns, pattern=','):

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)
    
    filtered = df_check[[df_index]]

    for site in df[df_index]:
        for column in columns[1:]:
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        #df_new = pd.merge(df_new, filtered, how='left', left_on='identification - site key', right_on=tw_index)
        df_new = df_new.set_index(df_index)
        #df_new = df_new[[status_col]+ df_new.columns[:-1].tolist()]
        #df_new = df_new[['identification - site key', status_col]]
        return df_new
    else: 
        print('\nNo one columns with incorrect Amount format!\n') 

lc_cols = ["FIN ID", 'LC Amount CZK\nyearly']  

df_lc_amount = check_amounts_lc(lc, "FIN ID", lc_cols)
df_lc_amount
# No errors


In [194]:
def check_lc_ta_dates(df,tw_index,start_date,end_date):
    df[start_date] = pd.to_datetime(df[start_date],errors='coerce')
    df[end_date] = pd.to_datetime(df[end_date],errors='coerce')
    filtered = df[df[start_date] > df[end_date]]
    if not filtered.empty:
        return filtered[[tw_index, start_date,end_date]]
    else:
        print('\nNo Errors Founded!\n')
        
df_lc_dates = check_lc_ta_dates(lc,"FIN ID", "Contr. Start date","Contr. 1st End")
df_lc_dates



No Errors Founded!



In [ ]:
lc_cols = ['FIN ID', 'Counterpart', 'LC Amount CZK\nyearly','Contr. Start date', 'Contr. 1st End']
path_lc_input = '/content/LC_Input_CzechRepublic_20210731.csv'
lc_old = '/content/LC_Input_CzechRepublic_20210630.csv'
lc_save = '/content/CZ_LC'
find_diffs_between_files(lc_old, path_lc_input, 'FIN ID', lc_cols, "", lc_save,'csv', 'lc')

In [13]:
def find_diffs_between_files_lc(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save, type_file='mix',kind='tw', kind_col='', k='', am_cols = [],\
                             cols_int=[], cols_date=[], sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist
     
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format
    
    def date_parser(df, column,format, type_date):
        if format == 'mix':
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
            return df[column]
        else:
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df


    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    am_cols = lower_str(am_cols)
    cols_int = lower_str(cols_int)
    cols_date = lower_str(cols_date)
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        #Ajustado só para CZ_Julho, tirei o encoding
        df_NEW = pd.read_csv(path_NEW).fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    else:
        col_order_lc = ["Contract ID - NEW","FIN ID","Contract Crncy","Contract Type","Freq.","Freq. Unit","Indexation",\
                        "Index upon request","Counterpart ID","Counterpart","LC Amount CZK\nyearly","Amount in EUR yearly (Actual)",\
                        "Payment terms Code","VAT Subject","% of VAT","Contr. Start date","Contr. 1st End","Contr. Term End",\
                        "Sublease Consession","Renewal Option","Unique Key","Count of Key","Count of ContrBct ",\
                        "Remarks - Consistency check","Residual Period in years","Scoping classification","DAS sites",\
                        "DAS ownership","31_12_9999","Unique Key1",'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021',\
                        "Check LC amount","Updated Item\nAmount yearly",'Contr. 1st End as of 31.05.2021','Contr. 1st End as of 30.06.2021',\
                        "Check Contr. 1st End","Updated Item\n1st End date","Comment","Comment date","Date"]
        col_order_lc = lower_str(col_order_lc)
        df_OLD = pd.read_csv(path_OLD, header=0, names=col_order_lc)
        #df_OLD = fit_df(df_OLD,bill_cols)
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = df_OLD.dropna(subset=[index_col], axis=0)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        df_OLD = df_OLD.sort_values(by='contract id - new')
        
        lista = []
        df_OLD['sites'] = df_OLD['contract id - new'].copy()
        for i in range(len(df_OLD['contract id - new'])):
            if str(df_OLD.iloc[i]['contract id - new']) in lista:            
                df_OLD.iloc[i]['sites'] = str(df_OLD.iloc[i]['contract id - new'])+"_"+str(lista.count(str(df_OLD.iloc[i]['contract id - new'])))
            else:
                df_OLD.iloc[i]['sites'] = str(df_OLD.iloc[i]['contract id - new'])+"_0"    
            lista.append(str(df_OLD.iloc[i]['contract id - new']))   

       

        #df_OLD['sites'] = df_OLD[index_col].astype(str)+df_OLD[kind_col].astype(str)+df_OLD[k].astype(str)+df_OLD['count of contrbct '].astype(str)+df_OLD["count of key"].astype(str)+df_OLD["counterpart"].astype(str)
        df_OLD = df_OLD.fillna('')

        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])

        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        #df_OLD = df_OLD.set_index('sites').fillna('')

        col_names = ['Contract ID - NEW', 'FIN ID', 'Contract Crncy', 'Contract Type', 'Freq.', 'Freq. Unit', 'Indexation',\
                'Index upon request', 'Counterpart ID', 'Counterpart', 'LC Amount CZK\nyearly', 'Amount in EUR yearly (Actual)', \
                'Payment terms Code', 'VAT Subject', '% of VAT', 'Contr. Start date', 'Contr. 1st End', 'Contr. Term End', \
                'Sublease Consession', 'Renewal Option', 'Unique Key', 'Count of Key', 'Count of ContrBct ', \
                'Remarks - Consistency check', 'Residual Period in years', 'Scoping classification', 'DAS sites', \
                'DAS ownership', '31_12_9999', 'Unique Key1', 'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021', \
                'Check LC amount', 'Updated Item\nAmount yearly', 'Contr. 1st End as of 31.05.2021', \
                'Contr. 1st End as of 30.06.2021', 'Check Contr. 1st End', 'Updated Item\n1st End date', 'Comment', \
                'Comment date', 'Date']
        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname,header=0, names=col_names, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW.reindex(columns=col_order_lc)
        df_NEW = df_NEW.dropna(subset=[index_col], axis=0)
        df_NEW = df_NEW.sort_values(by='contract id - new')

        df_NEW['sites'] = df_NEW[index_col].copy()
        df_NEW['sites'] = df_NEW['sites'].astype(str)
        df_NEW['sites'] = np.where(df_NEW['sites'].duplicated(keep=False), 
                      df_NEW['sites'] + df_NEW.groupby('sites').cumcount().add(1).astype(str),
                      df_NEW['sites'])
        print(df_NEW['sites'])
        #df_NEW['sites'] = [str(site)+"_"+str(i) for i, site in enumerate(df_NEW['contract id - new'])]

        #df_NEW['sites'] = df_NEW[index_col].astype(str)+df_NEW[kind_col].astype(str)+df_NEW[k].astype(str)+df_NEW['count of contrbct '].astype(str)+df_NEW["counterpart"].astype(str)+df_NEW["counterpart"].astype(str)
        
        df_NEW = replace_values(df_NEW, am_cols, 0)
        for col in am_cols:
            #df[col] = df[col].fillna(0)
            df_NEW[col] = df_NEW[col].astype(int).apply(lambda x: f'{x:,}')

        df_NEW = df_NEW.fillna('')

        df_NEW = replace_values(df_NEW,bill_cols)
        for i in cols_date:
            df_NEW[i] = date_parser(df_NEW, i, 'mix', "%d/%m/%Y")
        df_NEW = df_NEW.applymap(lambda x: x.encode('unicode_escape').decode('ISO-8859-1') if isinstance(x, str) else x)
        #print(df_NEW.info())
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos

        try:
            df_NEW = df_NEW.set_index('sites').fillna('')
        except:
            print('falha')
        #print(list(df_NEW.columns)==(df_OLD.columns))

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:

        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} old_file vs new_file.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                             end_color='00FF0000', fill_type='solid')
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)

    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

lc_col = ['FIN ID', 'Counterpart', 'LC Amount CZK\nyearly','Contr. Start date', 'Contr. 1st End']
path_lc_input = '/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx'
lc_old = '/content/LC_Input_CzechRepublic_20210731.csv'
lc_save = '/content/CZ_LC'
ta_tab = 'Final data_Lease'
dates_lc = ['Contr. Start date', 'Contr. 1st End', 'Contr. 1st End as of 31.05.2021', 'Contr. 1st End as of 30.06.2021']
interger_lc = []
amount_lc = ['LC Amount CZK\nyearly']
lc_bill_cols = ['Contract ID - NEW','Counterpart']
"""find_diffs_between_files(ta_old, path_ta_input, 'Code', ta_cols, \
                         status_col='', path_save=ta_save, type_file='mix',kind='ta',\
                         kind_col='tenant agreement id - new',k='service type',am_cols = amount_ta,\
                         cols_int=interger, cols_date=dates, sheetname=sheet)"""
find_diffs_between_files_lc(lc_old, path_lc_input, 'FIN ID', lc_col,status_col='', path_save=lc_save,type_file='mix', kind='ta',\
                            kind_col = 'contract id - new', k='contract type',am_cols=amount_lc,\
                           cols_int=[], cols_date=dates_lc, sheetname=ta_tab, skipr=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0        111530
1        111470
2        211800
3       1918801
4       1918802
         ...   
4671    ZSNA1G4
4672     300230
4673    7501002
4674    2290102
4675     241690
Name: sites, Length: 4676, dtype: object
[]


KeyboardInterrupt: ignored

In [ ]:
"""Verify before, cause you need to convert all values in cols for string format to check"""
!pip install unidecode
from unidecode import unidecode
def replace_values(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
    #lista = []

    df.fillna('', inplace=True)
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

    return df

"""Verify before, cause you need to convert all values in cols for string format to check"""
def date_parser(df, columns, t=1, format=1, type_dates='normal'):
    t_col = type_dates.lower()
    if format == 1:
        type_date = "%d/%m/%Y"
    else:
        type_date = "%d-%m-%Y"
    for column in columns:
        if t == 1:
            df[column] = pd.to_datetime(df[column], errors='coerce').fillna('')
            if t_col == 'mixed':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
        else:
            if t_col == 'mixed':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

#Columns to parser
col_order = ["towerdb owner", "identification - site key", "identification - code", "identification - fl",\
             "position - site name", "identification - other mno fl", "position - macro region", \
             "position - region", "position - municipality", "position - address", "position - latitude", \
             "position - longitude", "category - categorization by transmission sys (subcluster)", \
             "category - categorization by site type", "column1", "category - categorisation by inhabitants", \
             "other - ownership", "other - status", "column2", "column3", "column4", "column5", "column6", \
             "column7", "column8", "column9", "column10", "lease contract - current annual lease fee", \
             "lease contract - lease contract comment", "column11", "column12", "column13", "column14", \
             "tenants - anchor", "tenants - vf - ran sharing", "tenants - vf - passive", \
             "tenants - wind - ran sharing", "tenants - wind - passive", "tenants - cosmote - passive", \
             "tenants - ran sharing tenants", "tenants - passive tenants", "tenants - non-mno tenants", \
             "tenants - non-mno tenant name", "tenants - total tenants", "column15", \
             "ee - no. of 24hr generators", "ee - no. of standby generators", "kpi - sites", "kpi - tenants", \
             "msa - bts/replacement", "msa - bts commitment site", "column16", \
             "infrastructure type - technology", "infrastructure type - fibre / microwave", \
             "infrastructure type - floor space", "infrastructure type - tower height", "column17",\
             "notified-97 (gr use only)", "changes (gr use only)", "msa - billing trigger stop date", \
             "other - indoor vs outdoor", "skylon - category", "msa - standard configuration", \
             "msa - construction type", "msa - hub sites total mw diameter", "bp - consolidated classification",\
             "msa - critical site", "sensitive - due to", "sensitive - description", "sensitive - department", \
             "notified - department", "notified - due to", "msa - sensitive", "msa - notified", \
             "notified - status", "notified - details", "msa - rectifier", "msa - battery", "msa - aircon", \
             "msa - revenue generating", "msa - date of decommissioning", "flag indicating bts site",\
             "msa - billing trigger date", "msa - dg-shelter", "msa - critical site in excess of the 15.5% cap",\
             "msa - form of active sharing", "msa - start date for active sharing arrangement",\
             "msa - end date for active sharing arrangement", "column18", "msa - rfai date", \
             "msa - site acceptance date", "msa - sub-lease reason", "msa - sub-lease applies", \
             "msa - date of equipment removal (vf)", "msa - date of equipment removal (wh)"]

path_tw = "/content/GR_TowerDB Jun'21 FINAL (Billing version).xlsx"
sheet_tw = 'GR_TowerDB'
skipr = 0
skipc = 0

towerdb = read_files(path_tw, sheet_tw, skipr, skipc, 'Identification - Site Key')
towerdb.columns = lower_str(list(towerdb.columns))
towerdb = towerdb.reindex(columns = lower_str(col_order))


towerdb[tw_bts] = new_format(towerdb, tw_bts)

dates=['msa - start date for active sharing arrangement','msa - date of decommissioning',\
       'msa - end date for active sharing arrangement', 'msa - rfai date', 'msa - site acceptance date',\
       tw_doer_vf, tw_doer_wh]
date_parser(towerdb, [tw_bill, 'msa - billing trigger stop date'],2, 1,'normal')
date_parser(towerdb, dates,1, 1, 'mixed')

"""Datas a verificar no prx mês"""
#towerdb.loc[(towerdb[tw_bill]=='09/09/9999')&(towerdb[tw_bts]=='No'), tw_bts] = 'Yes'

# Para o caracter A (alpha)
towerdb[tw_index] = towerdb[tw_index].apply(unidecode)

#Sempre fazer o replace depois de todas as modificações
towerdb = replace_values(towerdb, lower_str(col_order))
#towerdb = towerdb.round(2)
#O Ficheiro da Grecia dá error na conversão dos caracteres  quando
towerdb.to_csv('/content/TowerDB_Greece_20210831.csv', index=False, encoding='cp1258')